####Mood Tracker Application

allows users to log their **daily experiences in various languages**, classify their mood based on their text entries, and receive motivational messages. The project utilizes a combination of **Hugging Face pipelines**, **OpenAI GPT-3.5-turbo API**, and **Gradio** for a seamless and interactive user interface.

**Authors :**


Shahad Mousa Albalawi
& Nojood Mohammed Alnahdi

####Dependencies

In [1]:
!pip install gradio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.1/18.1 MB 81.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.7/318.7 kB 27.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.0/94.0 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 13.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.3/10.3 MB 94.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.8/62.8 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.4/71.4 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/130.2 kB 11.9 MB/s eta 0:00:00
  Attempting uninstall: tomlkit
    Found existing

In [2]:
!pip install openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 374.1/374.1 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 17.9 MB/s eta 0:00:00


In [3]:
!pip install -q python-dotenv==1.0.0

In [4]:
from transformers import pipeline
import pandas as pd
from openai import OpenAI
import gradio as gr
import matplotlib.pyplot as plt
from dotenv import dotenv_values

#### Text Classification Hugging Face Pipeline

In [11]:
#This is a model for a multi-label classification task that classifies text into different emotions. It works only in English.
classifier = pipeline("text-classification", model="SamLowe/roberta-base-go_emotions")

#### Translation Hugging Face Pipeline

In [6]:
# This is a model for a translation task, designed to translate text.
# We use it to translate any non-English text into English, so the classifier can then classify the emotions.

translator = pipeline(task="translation", model="facebook/nllb-200-distilled-600M")
languages = {
    "English": "eng_Latn",
    "French": "fra_Latn",
    "Arabic": "arb_Arab",
    "Spanish": "spa_Latn",
    "German": "deu_Latn",
    "Chinese (Simplified)": "zho_Hans",
    "Hindi": "hin_Deva"
}

config.json:   0%|          | 0.00/846 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/2.46G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/189 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/564 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/4.85M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.3M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/3.55k [00:00<?, ?B/s]

#### openAI API

In [7]:
# prepare openAI client with our api key
env_values = dotenv_values("./app.env")
client = OpenAI(
    api_key= env_values['OPENAI_API_KEY'],)

#### Dataframe using pandas

In [8]:
# Create a DataFrame to store user entries and perform analysis.

structure = {
    'Date': [],
    'Text': [],
    'Mood': []
}
df = pd.DataFrame(structure)

#### Gradio Interface

In [9]:
# Take the text and its source language, translate it to English, so that the classifier can perform the task.
def translator_text(text, src_lang):
  translation = translator(text, src_lang=src_lang, tgt_lang="eng_Latn")
  return translation[0]['translation_text']

In [10]:
# Take all the inputs from the user, including the mood (result from the classifier), and append them to the DataFrame.
def appender(date, text, mood):
  global df
  new_row = pd.DataFrame({'Date': [date], 'Text': [text], 'Mood': [mood]})
  df = pd.concat([df, new_row], ignore_index=True)

In [14]:
def main(date, src_lang, text):

  # First: Translate the text to English if it is not already in English.
  if src_lang!= 'English':
    text = translator_text(text, languages[src_lang])

  # Second : Classify the text
  mood = classifier(text)[0]['label']

  # Third : Show a message to the user depending on how they feel.
  chat_completion = client.chat.completions.create(
      messages=[
          {
              "role": "user",
              "content": f"I feel{mood}, can you tell me a message, without any introductory phrase, just the message itself.",
          }
      ],
      model="gpt-3.5-turbo",
  )

  # Finally : Save to DataFrame
  appender(date, text, mood)

  #Highlighted the output utilizing 'HighlightedText' in gradio
  highlighted_mood = [(f"Today you're feeling", mood)]
  return highlighted_mood, chat_completion.choices[0].message.content

#Interface
demo = gr.Interface(
    fn=main,
    inputs=[gr.Textbox(label="Enter Date (YYYY-MM-DD)"), gr.Dropdown(choices=list(languages.keys()),label="Select a Language",value="English"), gr.Textbox(label="What's happened today?")],
    outputs=[gr.HighlightedText(label="Mood"), gr.Textbox(label="Message")],
    title = "Daily Journal",
    description=(
        "Capture your daily experiences, reflections, and insights in a personal journal.\n"
        "Log and monitor your mood daily to identify patterns and trends over time.\n"
        "Get inspirational or motivational messages each day."
    ),
    theme=gr.themes.Soft() # theme form gradio documentation
)

demo.launch(debug=True)

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://7b32c5592165a44c6c.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://7b32c5592165a44c6c.gradio.live


In [15]:
# This Gradio interface displays a bar chart of mood frequency, in addition to updating the DataFrame.
def analysis_weeks():
  global df
  mood_counts = df['Mood'].value_counts()
  plt.figure(figsize=(8, 4))
  plt.bar(mood_counts.index, mood_counts.values, color=['#ff9999', '#66b3ff', '#99ff99', '#ffcc99'])
  plt.title('Mood Frequency')
  plt.xlabel('Mood')
  plt.ylabel('Frequency')
  plt.tight_layout()

  # Save the plot
  plt.savefig('mood_plot.png')
  plt.close()
  return 'mood_plot.png', df

#Interface
demo = gr.Interface(
    theme=gr.themes.Soft(),
    fn=analysis_weeks,
    inputs= None,
    outputs=[gr.Image(type='filepath'), gr.Dataframe(label="Your data")],
    title = "Daily Journal",
    description=(
        "Capture your daily experiences, reflections, and insights in a personal journal.\n"
        "Click on \"Generate\" to view your mood analysis."
    ),
)

demo.launch(debug=True)

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://0686e13e3e8ba661e7.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://0686e13e3e8ba661e7.gradio.live
